Python поддерживает **наследование** (*inheritance*) это механизм, позволяющий создавать новый класс на основе существующего. Это позволяет строить иерархии классов – от общего к частному.

Класс, от которого наследуют, называется **родительским** (*parent class*), **базовым** (*base class*) или **суперклассом** (*superclass*). Класс-наследник называется **дочерним** (*child class*) или **подклассом** (*subclass*).

Подкласс наследует атрибуты суперкласса, но может их **переопределять** (*override*) и добавлять свои атрибуты. 

Создадим базовый класс:

In [6]:
class Cell:
    """Базовый класс биологической клетки"""
    
    def __init__(self, size, membrane_type="липидный бислой"):
        self.size = size  # размер в микрометрах
        self.membrane_type = membrane_type
        self.is_alive = True
    
    def divide(self):
        """Деление клетки"""
        print(f"Клетка размером {self.size} мкм делится")
        new_size = self.size / 2
        self.size = new_size    # исходная клетка уменьшается
        return Cell(new_size, self.membrane_type)
    
    def get_info(self):
        """Информация о клетке"""
        return f"Клетка: размер={self.size} мкм, мембрана={self.membrane_type}"

## Расширение базового класса

Создадим класс эпителиальной клетки, который наследуется от `Cell`. Добавляется атрибут класса `self.layer_type` – тип слоя (`"simple squamous"` означает, что клетка является однослойной плоской). Также добавляется метод объекта `secrete()`. Таким образом класс `EpithelialCell` расширяет (*extend*) базовый класс `Cell`.

In [7]:
class EpithelialCell(Cell):
    """Эпителиальная клетка – выстилает поверхности органов"""
    
    def __init__(self, size, layer_type="simple squamous"):
        super().__init__(size)
        self.layer_type = layer_type
    
    def secrete(self, substance):
        """Секреция веществ"""
        print(f"Эпителиальная клетка выделяет {substance}")

`super().__init__()` – вызов инициализатора родительского класса.

`EpithelialCell` автоматически получает поля объектов `size`, `membrane_type`, `is_alive` и методы `divide()`, `get_info()`.

In [8]:
EpithelialCell.__dict__

mappingproxy({'__module__': '__main__',
              '__doc__': 'Эпителиальная клетка – выстилает поверхности органов',
              '__init__': <function __main__.EpithelialCell.__init__(self, size, layer_type='simple squamous')>,
              'secrete': <function __main__.EpithelialCell.secrete(self, substance)>})

Создадим экземпляр однослойной кубической (`"simple cuboidal"`) эпителиальной клетки:

In [9]:
epithelial = EpithelialCell(15, "simple cuboidal")
epithelial.__dict__

{'size': 15,
 'membrane_type': 'липидный бислой',
 'is_alive': True,
 'layer_type': 'simple cuboidal'}

In [10]:
epithelial.get_info()

'Клетка: размер=15 мкм, мембрана=липидный бислой'

In [11]:
e2 = epithelial.divide()
type(e2)

Клетка размером 15 мкм делится


__main__.Cell

In [12]:
e2.get_info()

'Клетка: размер=7.5 мкм, мембрана=липидный бислой'

In [13]:
epithelial.get_info()

'Клетка: размер=7.5 мкм, мембрана=липидный бислой'

### Механизм работы `super()`

Может возникнуть закономерный вопрос: как функция `super()` определяет текущий класс, если мы не передаём ей никаких аргументов? В Python 3 вызов `super()` без аргументов эквивалентен `super(__class__, self)`, где `__class__` – это неявная ссылка на класс, в котором определён метод. Эта переменная создаётся на этапе компиляции: когда компилятор Python обнаруживает в теле метода обращение к `super()` или к `__class__`, он автоматически добавляет `__class__` как **свободную переменную замыкания** (*closure free variable*). 

In [14]:
class SubCell(Cell):
    def __init__(self):
        print(__class__)
        # super().__init__(15)
        super(__class__, self).__init__(15)
    
SubCell()

<class '__main__.SubCell'>


### Нарушение принципа подстановки Лисков

Обратите внимание, что метод `divide()` в базовом классе `Cell` возвращает объект типа `Cell`, а не тип вызывающего объекта. Это нарушает **принцип подстановки Лисков** (*Liskov Substitution Principle*, LSP) – один из принципов SOLID, согласно которому объекты дочернего класса должны быть взаимозаменяемы с объектами родительского класса без изменения корректности программы. В нашем случае при вызове `epithelial.divide()` мы ожидаем получить новую эпителиальную клетку, но получаем обычную `Cell`, теряя атрибут `layer_type`. Исправить это можно, заменив явное создание `Cell(...)` на `type(self)(...)`, что динамически определит класс объекта и вызовет соответствующий конструктор. Однако здесь возникает вторая проблема: конструкторы `Cell` и `EpithelialCell` имеют **разные сигнатуры** – базовый класс принимает `size` и `membrane_type`, а дочерний – `size` и `layer_type`. Это означает, что для корректной работы полиморфного метода `divide()` необходимо либо унифицировать сигнатуры конструкторов, либо переопределять метод `divide()` в каждом дочернем классе. 

## Переопределение методов

Дочерний класс может переопределить методы родителя (*Method Overriding*):

In [15]:
class RedBloodCell(Cell):
    """Эритроцит (красная кровяная клетка)"""
    
    def __init__(self, size=7):
        super().__init__(size)
        self.hemoglobin_count = 270_000_000  # молекул гемоглобина
    
    def get_info(self):
        """Переопределяем метод родителя"""
        base_info = super().get_info()  # получаем информацию от родителя
        return f"{base_info}, гемоглобин={self.hemoglobin_count} молекул"
    
    def divide(self):
        """Эритроциты не делятся!"""
        print("Эритроциты не способны к делению")
        return None

In [16]:
erythrocyte = RedBloodCell()
erythrocyte.get_info()

'Клетка: размер=7 мкм, мембрана=липидный бислой, гемоглобин=270000000 молекул'

In [17]:
isinstance(erythrocyte, RedBloodCell)

True

In [18]:
isinstance(erythrocyte, Cell)

True

In [19]:
RedBloodCell.__bases__

(__main__.Cell,)

In [20]:
Cell.__bases__

(object,)

Несмотря на то, что при определении класса `Cell` мы не задавали никаких классов для наследования, тем не менее, по умолчанию класс наследуется от базового класса `object`. Соответственно, в классе `Cell` содержатся не только поля и методы, которые мы в нем определили, но также поля и методы, которые определены в суперклассе `object`.

### Порядок разрешения методов (MRO)

Иногда в случае цепочки наследований сложно отследить в каком классе определен метод, вызываемый у объекта. Для выбора метода, который будет вызван у объект, класс которого может в общем случае наследовать методы с одним и тем же именем от разных классов, Python руководствуется определенным правилом – **порядком разрешения методов** (*Method Resolution Order*).

Рассмотрим абстрактный пример наследования:

In [21]:
class A:
    def f(self): print("A.f")

class B:
    def f(self): print("B.f")

class C(A, B):
    pass

c = C()
c.f()

A.f


При вызове метода `f()` класс `C` использует метод, определенный в классе `A`, так как он шел первым аргументом при определении класса `C`.

Поле `__mro__` класса содержит последовательность наследования в виде кортежа. Метод класса `mro()` также возвращает последовательность наследования, но в виде списка. Посмотрим на порядок наследования в классе `C`:

In [22]:
C.__mro__, C.mro()

((__main__.C, __main__.A, __main__.B, object),
 [__main__.C, __main__.A, __main__.B, object])

Напишем функцию, которая для заданного объекта и имени метода возвращает класс, в котором определен метод:

In [23]:
def findDefiningClass(obj, methodName):

    typeOfObject = type(obj)            # возвращает класс объекта
    for t in typeOfObject.mro():        # перебирается список иерархии классов;
        if methodName in t.__dict__:    # если имя метода обнаруживается в
            return t                    # словаре атрибутов класса t, возвращается t

In [24]:
findDefiningClass(c, 'f')

__main__.A